# 創建一個sqlite檔案，整理抓到的csv檔案

* 利用 glob及 pandas套件來讀取資料

* 建立一個 sqlite檔案 來存入資料庫

* 進一步整理為用個股來分的資料庫

## 1. 讀取我們抓到的資料
### 用 pandas一個一個讀入很沒有效率，使用 glob套件一次讀入所有的 csv檔案

In [1]:
import pandas as pd
import sqlite3
import glob

In [3]:
# glob 套件是用來找符合特定規則的文件名，跟我們用搜尋跳出來的結果差不多，這邊我們查詢附檔名為csv的檔案並存為一個列表的形式
